# Things to try:

## Join metadata into input data
Just join all metadata into one and also join it to the input comment string. 
Got about 77% acc max

## Join metadata for each example
##### Then pass everything after the LSTM  OR pass everything into the input
['kristiina', '11111111', '82.131.41.237'] becomes 'kristiina 11111111 82.131.41.237'
Then create 2 embedding layers for both the meta and comment data.
Got about 74% acc max

## Keep all metadata separate, tokenize all of them separately and then join later
##### Then pass everything after the LSTM  OR pass everything into the input

## Do both, join and input join
Add metadata to input, then feed it in later as well. Doing this now.
Got almost 78% with a CNN

In [1]:
import numpy as np 

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Input, concatenate, Dropout, SpatialDropout1D, Flatten, BatchNormalization, Conv1D, MaxPooling1D
from keras.utils.np_utils import to_categorical
from keras import regularizers

from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from process_data import process_document
import json

C:\Users\Ranet\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
data = json.load(open('processed_text_with_metadata.json', encoding='UTF8'))

X = data['X']
y = data['y']
X_metadata = data['X_metadata']
X_val = data['X_val']
y_val = data['y_val']
X_val_metadata = data['X_val_metadata']

In [3]:
#105 = fine
index = 1
print(X[index])
print(X_metadata[index])
print(y[index])

kurat. b- proovi järel tuleb avaldada. täiesti haige,et sporti ka ronib haige privaatsuse teema. astu tagasi mardna. silbaum ju käis läbi. kurat ma ei tea kuidas see privaatsuse katk küll likvideerida. teised ootavad b- proovini. jefimova vilistati kuna see polnud esimene kord.
['kristiina', '11111111', '82.131.41.237']
1


In [13]:
iter_x = [[len(x[i]) for i, _ in enumerate(x)] for x in X_metadata]
print('mean lens: ', [np.mean(iter_x[0]), np.mean(iter_x[1]), np.mean(iter_x[2])])
print('max lens: ', max(iter_x))

# join ip
for i in range(len(X_metadata)):
    ip = ''.join(X_metadata[i][2].split('.'))
    parent = X_metadata[i][1]
    X_metadata[i][1] = ip
    X_metadata[i][2] = parent

for i in range(len(X_val_metadata)):
    ip = ''.join(X_val_metadata[i][2].split('.'))
    parent = X_val_metadata[i][1]
    X_val_metadata[i][1] = ip
    X_val_metadata[i][2] = parent
    

print(X_metadata[0])

X_metadata_joined = [' '.join(x) for x in X_metadata]
X_val_metadata_joined = [' '.join(x) for x in X_val_metadata]

print(X_metadata_joined[0])
print(X_val_metadata_joined[0], '\n')


# Join metadata to comment
X_with_meta = X
X_val_with_meta = X_val
for i, x in enumerate(X_with_meta):
    X_with_meta[i] =  (X_metadata[i][0] + ' ' + X_metadata[i][1]) + ' ' + X_with_meta[i] +  ' ' + X_metadata[i][2]

for i, x in enumerate(X_val_with_meta):
    X_val_with_meta[i] =  X_val_metadata_joined[i] + ' ' + X_val_with_meta[i]
    
print(X_with_meta[0])
    



mean lens:  [21.666666666666668, 9.0, 7.666666666666667]
max lens:  [249, 10, 8]
['aga', 'ei tegu,ei näguole nii tark kui tahadei sobi', '195250187225']
aga ei tegu,ei näguole nii tark kui tahadei sobi 195250187225
hihi milleks peabki armu andma,kas need kes tahavad armu saada antsid armu oma ohvriletubli ilves,veel vastupidi peaks olema ja neile aastaid just juurde panema 2177147107 

aga ei tegu,ei näguole nii tark kui tahadei sobi aga 195250187225 ega ka mamma tarja ju missimõõtu polnud. ei tegu,ei nägu...ole nii tark kui tahad...ei sobi 195250187225


In [14]:
vocab_size = 100000
seq_len = 1000

def tokenization_processing(x_to_tok, x_val_to_tok):
    tokenizer = Tokenizer(num_words=vocab_size,
                                       filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n',
                                       lower=True,
                                       split=" ",
                                       char_level=False,
                                       oov_token=None)

    tokenizer.fit_on_texts(x_val_to_tok + x_to_tok)
    print(len(tokenizer.word_index))

    X_tokenized = tokenizer.texts_to_sequences(x_to_tok)
    X_val_tokenized = tokenizer.texts_to_sequences(x_val_to_tok)

    X_tokenized = pad_sequences(X_tokenized, maxlen=seq_len)
    X_val_tokenized = pad_sequences(X_val_tokenized, maxlen=seq_len)
    
    print(x_val_to_tok[0])
    print(X_val_tokenized[0])
    print(x_to_tok[0])
    print(X_tokenized[0])
    
    return [X_tokenized, X_val_tokenized]

X_tokenized, X_val_tokenized = tokenization_processing(X_with_meta, X_val_with_meta)

X_metadata_tokenized, X_val_metadata_tokenized = tokenization_processing(X_metadata_joined, X_val_metadata_joined)

y_oh = to_categorical(y, 2)
y_val_oh = to_categorical(y_val, 2)

80368
hihi milleks peabki armu andma,kas need kes tahavad armu saada antsid armu oma ohvriletubli ilves,veel vastupidi peaks olema ja neile aastaid just juurde panema 2177147107 hihi 2177147107 milleks peabki armu andma,kas need kes tahavad armu saada antsid armu oma ohvrile.tubli ilves,veel vastupidi peaks olema ja neile aastaid just juurde panema ma ytleks sulle hoopis nii .. "kuul pähe"!
olen vist liiga palju näinud usa krimilahendusi ..
:)
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     

36535
hihi milleks peabki armu andma,kas need kes tahavad armu saada antsid armu oma ohvriletubli ilves,veel vastupidi peaks olema ja neile aastaid just juurde panema 2177147107
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0 

In [15]:
print(X_with_meta[0])
print(X_tokenized[0])

print(X_metadata_joined[0])
print(X_metadata_tokenized[0])

print()

aga ei tegu,ei näguole nii tark kui tahadei sobi aga 195250187225 ega ka mamma tarja ju missimõõtu polnud. ei tegu,ei nägu...ole nii tark kui tahad...ei sobi 195250187225
[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0  

In [16]:
embed_dim = 100 #30 and 60 for meta

inputs = Input(shape=(seq_len,), name='inputs')
x = Embedding(vocab_size, embed_dim, input_length=seq_len)(inputs)

metadata_inputs = Input(shape=(seq_len,), name='metadata_inputs')
metadata_embedding = Embedding(vocab_size, 60, input_length=seq_len)(metadata_inputs)

x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling

x = Flatten()(x)
metadata_embedding = Flatten()(metadata_embedding)
x = concatenate([x, metadata_embedding])

x = Dense(128, activation='relu')(x)
output = Dense(2, activation='softmax')(x)




model = Model(inputs=[inputs, metadata_inputs], outputs=[output])
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [17]:
num_epochs = 50
batch_size = 256

model.fit([X_tokenized, X_metadata_tokenized], y_oh, epochs=num_epochs, batch_size=batch_size, verbose=1, validation_data=([X_val_tokenized, X_val_metadata_tokenized], y_val_oh))
# print(model.summary())

Train on 9944 samples, validate on 2487 samples
Epoch 1/50
9944/9944 [==============================] - 5s 540us/step - loss: 0.7611 - acc: 0.5255 - val_loss: 0.6708 - val_acc: 0.5657
Epoch 2/50
9944/9944 [==============================] - 4s 436us/step - loss: 0.5654 - acc: 0.7536 - val_loss: 0.5341 - val_acc: 0.7209
Epoch 3/50
9944/9944 [==============================] - 4s 441us/step - loss: 0.3110 - acc: 0.8921 - val_loss: 0.5191 - val_acc: 0.7519
Epoch 4/50
9944/9944 [==============================] - 4s 442us/step - loss: 0.1064 - acc: 0.9673 - val_loss: 0.5748 - val_acc: 0.7724
Epoch 5/50
9944/9944 [==============================] - 4s 439us/step - loss: 0.0444 - acc: 0.9864 - val_loss: 0.7759 - val_acc: 0.7411
Epoch 6/50
9944/9944 [==============================] - 4s 442us/step - loss: 0.0286 - acc: 0.9915 - val_loss: 0.7735 - val_acc: 0.7531
Epoch 7/50
1536/9944 [===>..........................] - ETA: 3s - loss: 0.0181 - acc: 0.9941

KeyboardInterrupt: 

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X_with_meta, X_val_with_meta)
X_train_counts.shape

from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(9944, 70320)

In [19]:
from sklearn.pipeline import Pipeline
from sklearn.naive_bayes import MultinomialNB
text_clf = Pipeline([('vect', CountVectorizer(stop_words='english')),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultinomialNB()),
])
text_clf = text_clf.fit(X_with_meta, y)
text_clf.score(X_val_with_meta, y_val)

0.7800562927221552